# MazeDrone with TF Agents - 5001 Testing Trained Agents

In [1]:
from __future__ import absolute_import, division, print_function

import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'




import matplotlib.pyplot as plt
import numpy as np
import PIL.Image

import tensorflow as tf
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.utils import common

import sys
import gym

from resources import build_agent, TrainingSession
import pandas as pd
import gc


2025-02-16 01:26:00.443822: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-16 01:26:00.448143: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-16 01:26:00.521182: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-16 01:26:01.679236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

min_combination = 2 # combination 2
min_run = 1
max_run = 1


num_iterations = 20_000 # @param {type:"integer"}

initial_collect_steps = 64  # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
#replay_buffer_max_length = 100000  # @param {type:"integer"}
replay_buffer_max_length = 100  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 100  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 100  # @param {type:"integer"}

# Agent fully connected layer params 
fc_layer_params = (200,) 

# File's name
description = "5001"

maze_size = 3

rewards = []
rewards.append({
    'destroyed': -10.,
    'stuck': - 6.,
    'reached': 10.,
    'standard': -1.
})
rewards.append({
    'destroyed': -10.,
    'stuck': -11.,
    'reached': 10.,
    'standard': -1.
})
rewards.append({
    'destroyed': -10.,
    'stuck': -15.,
    'reached': 10.,
    'standard': -1.
})


sys.path.append('/home/naski/Documents/dev/maze_drone_v02')
import gym_maze # Esta linha precisa estar após o PATH

# Importing custom environment
env_name = 'maze-v0'
env = suite_gym.load(env_name)

# Testing
env.reset()

train_py_env = suite_gym.load(env_name)
# Converts environments, originally in pure Python, to tensors (using a wrapper)
train_env = tf_py_environment.TFPyEnvironment(train_py_env)





# CREATING/RESETING THE AGENT
agent = build_agent(fc_layer_params, env, learning_rate, train_env)
agent.initialize()

# GENERATE TRAINING SESSION
session = TrainingSession(description, maze_size, env_name, rewards[0], agent, collect_steps_per_iteration, 
                        1, eval_interval, replay_buffer_max_length, num_eval_episodes)

# TRAINING
step_log, returns, finished, crashed, stucked, steped, _, replay_buffer, _ = session.train()

# CLEAR MEMORY
del(session)



Rewards Updated: {'destroyed': -10.0, 'stuck': -6.0, 'reached': 10.0, 'standard': -1.0}
Rewards Updated: {'destroyed': -10.0, 'stuck': -6.0, 'reached': 10.0, 'standard': -1.0}
Mode set: 1
Mode set: 1
Maze site set: 3x3
Maze site set: 3x3
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Rewards: {'destroyed': -10.0, 'stuck': -6.0, 'reached': 10.0, 'standard': -1.0}
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
learning_rate inside: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>


## Agent 1 - 2001

In [3]:
print("============================================================== Loading Checkpoint:")

print(agent.train_step_counter)

checkpoint_dir = './checkpoint/phase-3-1-tuning'
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=agent.train_step_counter
)

print(agent.train_step_counter)


train_checkpointer.initialize_or_restore()
    
print(agent.train_step_counter)

agent._optimizer.learning_rate = learning_rate
print(agent._optimizer.learning_rate)

============================================================== Loading Checkpoint:
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=13600>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=13600>
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>


In [4]:
from resources import compute_logs

In [5]:
print("========================================================================== Testing:")

eval_py_env = suite_gym.load(env_name)
eval_py_env.update_rewards(rewards[1]['destroyed'], rewards[1]['stuck'], rewards[1]['reached'], rewards[1]['standard'])
eval_py_env.set_mode(0) # walls
eval_py_env.set_size(maze_size)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

avg_return, finished_percentage, crashes_counter, stuck_counter, avg_steps = compute_logs(eval_env, agent.policy, rewards[1], 1000)

print(avg_return, finished_percentage, crashes_counter, stuck_counter, avg_steps)

========================================================================== Testing:
Rewards Updated: {'destroyed': -10.0, 'stuck': -11.0, 'reached': 10.0, 'standard': -1.0}
Mode set: 0
Maze site set: 3x3
1.05 0.884 0 0 8.674


In [6]:
from resources import create_policy_eval_video

# Reset the environment.
time_step = train_py_env.reset()
train_py_env.set_mode(0)
train_py_env.set_size(maze_size)


# Gera video da politica do agente treinado até então
create_policy_eval_video(agent.policy, f"trained-agent-{description}1", train_env, train_py_env)

Mode set: 0
Maze site set: 3x3


## Agent 2 - 2002

In [7]:
print("============================================================== Loading Checkpoint:")

print(agent.train_step_counter)

checkpoint_dir = './checkpoint/phase-3-2-tuning'
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=agent.train_step_counter
)

print(agent.train_step_counter)


train_checkpointer.initialize_or_restore()
    
print(agent.train_step_counter)

agent._optimizer.learning_rate = learning_rate
print(agent._optimizer.learning_rate)

============================================================== Loading Checkpoint:
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=13600>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=25000>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=25000>
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>


In [8]:
print("========================================================================== Testing:")

eval_py_env = suite_gym.load(env_name)
eval_py_env.update_rewards(rewards[2]['destroyed'], rewards[2]['stuck'], rewards[2]['reached'], rewards[2]['standard'])
eval_py_env.set_mode(0) # walls
eval_py_env.set_size(maze_size)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

avg_return, finished_percentage, crashes_counter, stuck_counter, avg_steps = compute_logs(eval_env, agent.policy, rewards[1], 1000)

print(avg_return, finished_percentage, crashes_counter, stuck_counter, avg_steps)

========================================================================== Testing:
Rewards Updated: {'destroyed': -10.0, 'stuck': -15.0, 'reached': 10.0, 'standard': -1.0}
Mode set: 0
Maze site set: 3x3
1.505 0.895 0 0 8.34


In [9]:
# Reset the environment.
time_step = train_py_env.reset()
train_py_env.set_mode(0)
train_py_env.set_size(maze_size)


# Gera video da politica do agente treinado até então
create_policy_eval_video(agent.policy, f"trained-agent-{description}2", train_env, train_py_env)

Mode set: 0
Maze site set: 3x3


## Agent 3 - 3001

In [10]:
print("============================================================== Loading Checkpoint:")

print(agent.train_step_counter)

checkpoint_dir = './checkpoint/phase-3-3-tuning'
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=agent.train_step_counter
)

print(agent.train_step_counter)


train_checkpointer.initialize_or_restore()
    
print(agent.train_step_counter)

agent._optimizer.learning_rate = learning_rate
print(agent._optimizer.learning_rate)

============================================================== Loading Checkpoint:
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=25000>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=14000>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=14000>
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>


In [11]:
print("========================================================================== Testing:")

eval_py_env = suite_gym.load(env_name)
eval_py_env.update_rewards(rewards[1]['destroyed'], rewards[1]['stuck'], rewards[1]['reached'], rewards[1]['standard'])
eval_py_env.set_mode(0) # walls
eval_py_env.set_size(maze_size)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

avg_return, finished_percentage, crashes_counter, stuck_counter, avg_steps = compute_logs(eval_env, agent.policy, rewards[1], 1000)

print(avg_return, finished_percentage, crashes_counter, stuck_counter, avg_steps)

========================================================================== Testing:
Rewards Updated: {'destroyed': -10.0, 'stuck': -11.0, 'reached': 10.0, 'standard': -1.0}
Mode set: 0
Maze site set: 3x3
2.15 0.908 0 9 7.748


In [16]:
# Reset the environment.
time_step = train_py_env.reset()
train_py_env.set_mode(0)
train_py_env.set_size(maze_size)


# Gera video da politica do agente treinado até então
create_policy_eval_video(agent.policy, f"trained-agent-{description}3", train_env, train_py_env)

Mode set: 0
Maze site set: 3x3


## Agent 4 - 3002

In [13]:
print("============================================================== Loading Checkpoint:")

print(agent.train_step_counter)

checkpoint_dir = './checkpoint/phase-3-4-tuning'
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=agent.train_step_counter
)

print(agent.train_step_counter)


train_checkpointer.initialize_or_restore()
    
print(agent.train_step_counter)

agent._optimizer.learning_rate = learning_rate
print(agent._optimizer.learning_rate)

============================================================== Loading Checkpoint:
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=14000>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=25200>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=25200>
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>


In [14]:
print("========================================================================== Testing:")

eval_py_env = suite_gym.load(env_name)
eval_py_env.update_rewards(rewards[2]['destroyed'], rewards[2]['stuck'], rewards[2]['reached'], rewards[2]['standard'])
eval_py_env.set_mode(0) # walls
eval_py_env.set_size(maze_size)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

avg_return, finished_percentage, crashes_counter, stuck_counter, avg_steps = compute_logs(eval_env, agent.policy, rewards[1], 1000)

print(avg_return, finished_percentage, crashes_counter, stuck_counter, avg_steps)

========================================================================== Testing:
Rewards Updated: {'destroyed': -10.0, 'stuck': -15.0, 'reached': 10.0, 'standard': -1.0}
Mode set: 0
Maze site set: 3x3
-1.686 0.833 0 0 10.779


In [15]:
# Reset the environment.
time_step = train_py_env.reset()
train_py_env.set_mode(0)
train_py_env.set_size(maze_size)


# Gera video da politica do agente treinado até então
create_policy_eval_video(agent.policy, f"trained-agent-{description}4", train_env, train_py_env)

Mode set: 0
Maze site set: 3x3
